In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = ""
api_key = os.getenv('MISTRAL_API_KEY')

In [ ]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

model = init_chat_model("mistral-large-latest", model_provider="mistralai")
search = TavilySearchResults(max_results=2)
tools = [search] # All tools in list

#bind tools with model
model_with_tools = model.bind_tools(tools)

In [ ]:
#Invoke model with message
response = model_with_tools.invoke([HumanMessage(content="Hi there!")])

print(f"Response by Model- ContentString : {response.content}")
print(f"Response by Tavily- ToolCalls : {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Delhi?")])

print(f"Response by Model- ContentString: {response.content}")
print(f"Response by Tavily- ToolCalls : {response.tool_calls}")
# Model Response to call tools
#This isn't calling that tool yet - it's just telling us to. 
#In order to actually call it, we'll want to create our agent.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)
# Use the agent
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi Model!")]})

response["messages"]

In [ ]:
#Streaming

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi there!")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what's the weather where in Delhi?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# For Memory Persistence, keeping conversation history

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

#Agent will have memory now
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im joy!")]}, config
):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")